# Hough lines

In [ ]:
from tip_detection import hough_lines_image, skeletonize
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np
import pandas as pd
import math

path = os.chdir('C:\\Users\\z004b1tz\\Desktop\\Master Thesis Project\\Binary segmentation\\Dataset\\output\\Skeletonized')
ground_truth_path = 'C:\\Users\\z004b1tz\\Desktop\\Master Thesis Project\\Binary segmentation\\Dataset\\output\\predictions\\predictions\\'

final = []

for file in os.listdir(path):
    if file.endswith('.PNG'):
        filename = os.fsdecode(file)
        ground = filename.replace('.PNG', '.raw.csv')
        image = cv2.imread(filename)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        lines = cv2.HoughLinesP(gray, rho=1, theta=np.pi/180, threshold=30, minLineLength=20, maxLineGap=60)
    
        for points in lines:
            # Extracted points nested in the list
            x1,y1,x2,y2=points[0]
            # Draw the lines joing the points On the original image
            cv2.line(image,(x1,y1),(x2,y2),(0,255,0),2)
        
        cells = pd.read_csv(ground_truth_path+ground, header=None , engine = 'python')
        gt_points = []
        for idx,row in cells.iterrows():
            x_start = row.iloc[0]
            y_start = row.iloc[1]
            x_end = row.iloc[2]
            y_end = row.iloc[3]
            gt_points.append([x_start, y_start])

        # get starting points of detected lines
        detected_points = []
        for line in lines:
            detected_points.append([line[0][0], line[0][1]])
            detected_points.append([line[0][2], line[0][3]])

        # calculate error between detected points and ground truth
        distance_errors = []
        for point in detected_points:
            for gt_point in gt_points:
                distance_error = np.sqrt((point[0] - gt_point[0]) ** 2 + (point[1] - gt_point[1]) ** 2)
                distance_errors.append(distance_error)

        # remove duplicates from errors list
        errors = list(set(distance_errors))
        # sort errors in ascending order
        errors.sort()

        err = []
        # get the number of errors with the minimum error
        num_of_detected_points_with_min_errors = len(gt_points)
        detected_points_with_min_errors = []
        angle_errors = []
        for gt_point in gt_points:
            for point in detected_points:
                error = np.sqrt((point[0] - gt_point[0]) ** 2 + (point[1] - gt_point[1]) ** 2)
                detected_angle = math.atan2(min_point[1], min_point[0])
                angle_error = round(abs(detected_angle - gt_angle),4)
                angle_errors.append(angle_error)
                err.append(error)
                detected_points_with_min_errors.append(point)
                detected_points.remove(point)

        result = {
            'file_name': filename,
            'ground_truth_points': gt_points,
            'detected_points_with_min_errors': detected_points_with_min_errors,
            'distance_error': err,
            'angle_error' : angle_errors
        }
        final.append(result)

        hough_lines_image(os.path.join(path, filename), os.path.join(path, ground))
        print(result)
        plt.imshow(image, cmap='gray')
        plt.show()

In [ ]:
import pandas as pd

df = pd.DataFrame(final)
mean_distance_error = np.mean(df['distance_error'].apply(lambda x: np.mean(x)))
print(mean_distance_error*(20/976))

# Detect the connected components

In [ ]:
from tip_detection import tip_detector_right
import os
import pandas as pd

path = 'C:\\Users\\z004b1tz\\Desktop\\Master Thesis Project\\5 Layer Binary UNET\\Dataset\\output\\predictions\\predictions 700x700'

result = {'filename':[], 'detected_points':[]} 

for file in os.listdir(path):
    if file.endswith('.PNG'):
        filename = os.fsdecode(file)
        ground = filename.replace('.PNG', '.raw.csv')
        _, points = tip_detector_right(path+'\\'+filename)
        result['filename'].append(ground)
        result['detected_points'].append(points)

In [ ]:
df = pd.DataFrame(result)
df

In [ ]:
d = {i: [] for i in df['filename'].unique()}
df.apply(lambda x: d[x[0]].append(x[1]), axis=1)
d = {key: value for key, value in d.items() if value[0]}
final = list(zip(*list(d.items())))

dic = {'filenames': final[0], 'detected_points': final[1]}
df2 = pd.DataFrame(dic)
df2

In [ ]:
true, false = 0,0

for file in df2['filenames']:
    cells = pd.read_csv('C:\\Users\\z004b1tz\\Desktop\\Master Thesis Project\\ground\\'+file, header=None , engine = 'python')
    if len(cells) == len(df2.loc[df2["filenames"] == file, "detected_points"].iloc[0]):
        true += 1
    else:
        false += 1

In [ ]:
true/(false+true)

In [ ]:
import numpy as np
import os
import pandas as pd

path_to_ground_truth = 'C:\\Users\\z004b1tz\\Desktop\\Master Thesis Project\\ground'

def calculate_errors(ground_truth_points, detected_points):
    # Calculate the errors
    mse = np.sqrt(np.mean((ground_truth_points - detected_points)**2))
    mae = np.mean(np.abs(ground_truth_points - detected_points))
    if len(ground_truth_points.shape) == 1:
        manhattan = np.mean(np.abs(ground_truth_points - detected_points))
    else:
        manhattan = np.mean(np.sum(np.abs(ground_truth_points - detected_points), axis=1))
    return [mse, mae, manhattan]


error = {'filename':[], 'error':[]}
for i, row in df.iterrows():
    # Get the filename and detected points
    filename = row['filename']
    detected_points = row['detected_points']
    num_detected_points = len(detected_points)
    min_errors = [float('inf')] * num_detected_points

    # Read the ground truth csv file
    ground_truth = os.path.join(path_to_ground_truth, filename)
    cells = pd.read_csv(ground_truth, header=None, engine='python')
    errors = [[] for _ in range(num_detected_points)]
    for idx, row in cells.iterrows():
        ground_truth_point = np.array([row.iloc[0], row.iloc[1]])
        for j, detected_point in enumerate(detected_points):
            error = calculate_errors(ground_truth_point, np.array(detected_point))
            for k in range(len(error)):
                if error[k] < min_errors[j]:
                    min_errors[j] = error[k]
                errors[j].append(error[k])
    print(filename)
    print("Errors:", errors)

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

path = 'C:\\Users\\z004b1tz\\Desktop\\Master Thesis Project\\Multi-class UNET\\Dataset\\output\\erosion\\01840_0025_976x976_16bpp.PNG'

image = cv2.imread(path)

_, thresh = cv2.threshold(image,128, 255, cv2.THRESH_BINARY)

print(np.unique(thresh))
plt.imshow(thresh, cmap='gray')